针对RPM产品，给定一个问题和解决方案，先扩张问题描述，然后反思这个更新的问题描述是否与原始版本保持一致。接着，根据更新的问题描述和原始的解决方案来生成多个解决方案，
再结合多个解决方案来生成最终的解决方案。目前的重点是完善问题描述和解决方案。

Use GPT-4

In [1]:
import os
from dotenv import load_dotenv
from datetime import datetime

import openai
from IPython.display import HTML, display
from ipywidgets import widgets
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import DirectoryLoader
from langchain.vectorstores import Chroma

from langchain.chains import LLMChain
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import ChatPromptTemplate
from langchain.memory import ConversationBufferMemory, ConversationSummaryBufferMemory


load_dotenv("../openai.env")
llm = ChatOpenAI(model_name="gpt-4-0613", temperature=0.7, max_tokens=2000)

In [3]:
import pandas as pd

ideas = {}
df = pd.read_excel('patents/rpm-patents-0628d.xlsx')
for i, row in df.iterrows():
    pno, question, solution = row['no.'], row['question'], row['solution']
    if pd.isna(question) or pd.isna(solution): continue
    ideas[pno] = {'question': question, 'solution': solution}
print(ideas.keys())

dict_keys(['P73', 'P74', 'P75'])


In [4]:
question = ideas['P73']['question']
print(question)

使用GPT可以生成规模并不复杂的代码，包括RPA代码，一般的方法是设置一个比较详细的生成提示词，但是这样的生成过程难以控制，比如使用了不存在的库函数，或者缺少必要的数据等等，也难以保证生成的流程可以正常执行。


In [8]:
template_rewriteq = '''你是一个专业的软件方法方向的专利代理，你的任务是帮助专利发明人完善发明点设计。
请根据发明人提供的简要描述和以下规则，重新描述专利发明点要解决的问题。
1，待解决的问题应用于什么样的业务场景？请给出一个简单且有代表性的例子，方便理解。
2，这类问题的一般解决思路是什么？这样的解决思路存在什么样的功能或者性能上的弱点或者缺陷？请给出技术上的理由。
3，为什么要解决这个问题？它有多大的现实价值？请结合业务场景和技术特征进行分析。

待解决问题的简单描述: {raw_question}

请根据以上规则要求，以更完整、更详细的方式重新描述要解决的问题（总长度不超过2000字）：'''

prompt_rewriteq = PromptTemplate(input_variables=['raw_question'], template=template_rewriteq)
rewriteq_chain = LLMChain(llm=llm, prompt=prompt_rewriteq, output_key='question')

In [9]:
answer = rewriteq_chain.run(question)
print(answer)

待解决的问题应用于自动化编程的业务场景。例如，某个企业希望使用机器学习工具，如GPT，自动生成一些用于自动化其业务流程的RPA（Robotic Process Automation）代码。这些业务流程可能包括自动化报告生成，自动化数据分析等。在这个例子中，企业可能会提供一些生成提示词（如"生成一个自动化报告生成的RPA代码"）来指导GPT生成代码。然而，这样的方法存在一些问题，比如GPT可能会生成一些调用了不存在的库函数的代码，或者生成的代码缺少必要的数据，或者即使代码在语法上没有问题，也可能因为逻辑错误无法正常执行。

一般来说，解决这类问题的思路是通过设置更详细的生成提示词来指导GPT生成代码。例如，使用"生成一个自动化报告生成的RPA代码，该代码需要使用库函数X，Y，Z并且需要数据A，B，C"作为生成提示词。然而，这样的解决思路存在一些弱点和缺陷。首先，即使生成提示词非常详细，也无法保证GPT生成的代码能够正常执行。因为GPT生成代码的过程是基于其在大量代码上训练得到的模型，而这个模型只能学习到代码的一般模式，无法理解代码的具体逻辑。因此，生成的代码可能在语法上没有问题，但是在逻辑上存在错误。其次，设置详细的生成提示词需要专门的编程知识，这对于非专业的编程人员来说是一项困难的任务。

解决这个问题有很大的现实价值。首先，如果能够解决这个问题，那么非专业的编程人员也可以通过设置生成提示词来生成代码，这将大大降低自动化编程的门槛。其次，如果生成的代码可以保证正常执行，那么可以大大提高自动化编程的效率和可靠性。结合业务场景，例如上面提到的企业，如果能够解决这个问题，那么企业就可以通过生成提示词来自动生成RPA代码，从而自动化其业务流程，这将大大提高企业的工作效率。从技术特征来看，如果能够解决这个问题，那么就可以充分发挥GPT在自动化编程方面的潜力，从而推动自动化编程技术的发展。


In [ ]:
template_rewriteq = '''你是一个专业的软件方法方向的专利代理，你的任务是帮助专利发明人完善发明点设计。
请根据发明人提供的简要描述和以下规则，重新描述专利发明点要解决的问题。
1，待解决的问题应用于什么样的业务场景？请给出一个简单且有代表性的例子，方便理解。
2，这类问题的一般解决思路是什么？这样的解决思路存在什么样的功能或者性能上的弱点或者缺陷？请给出技术上的理由。
3，为什么要解决这个问题？它有多大的现实价值？请结合业务场景和技术特征进行分析。
    你能描述你的发明解决了什么具体的软件或计算问题吗？

    那些用户、软件系统或计算环境会遇到这个问题？

    你能举一个这个软件问题通常出现的例子吗？

    这个问题如何影响软件系统或应用的操作、效率或用户体验？

    目前存在哪些针对这个问题的软件或算法解决方案？他们在哪些方面不够完善、高效或者用户友好？

    在软件开发或计算技术领域中，有哪些最近或即将发生的变化使得这个问题变得特别相关或紧迫？

    你能量化这个问题吗？有没有任何统计数据或者度量标准能够说明其对计算性能、系统效率或用户体验的影响？

    这个问题如何影响不同的利益相关者（例如：开发人员、终端用户、系统管理员、业务所有者）？

    这个软件问题是被广泛认知的，还是说你的发明揭示了它？

    解决这个问题有没有任何特定的软件、硬件或系统约束？
待解决问题的简单描述: {raw_question}

请根据以上规则要求，以更完整、更详细的方式重新描述要解决的问题（总长度不超过2000字）：'''

prompt_rewriteq = PromptTemplate(input_variables=['raw_question'], template=template_rewriteq)
rewriteq_chain = LLMChain(llm=llm, prompt=prompt_rewriteq, output_key='question')

In [3]:
template_commentq = '''你是一个专业的流程挖掘产品经理，请评估更新版本的问题描述是否存在不完善的地方。
如果存在不完善的地方，请重新描述一下。如果没有，可以直接复用原来的问题描述。

原始版本的问题描述: {raw_question}
更新版本的问题描述: {question}

请在更新版本的基础上，回答以下问题：
1，更新版本是否表达了与原始版本相同的问题？如果有不一致的地方，请修正一下；
2，更新版本是否增加了更多的技术细节、应用场景，而不只是引用或者复述原始版本？如果未增加更多的细节，请补充；
3，更新版本是否解决了原始版本中未清晰表达的关键内容？如果没有很好的解决，请重新更正；
4，更新版本都包含了哪些核心概念和术语？请解释这些概念和术语，以及它们之间可能的逻辑关系。

对更新版本的评估（总长度不超过2000字）：'''

prompt_commentq = PromptTemplate(input_variables=['raw_question', 'question'], template=template_commentq)
commentq_chain = LLMChain(llm=llm, prompt=prompt_commentq, output_key='commentq')

In [4]:
template_candidates = '''你们是三个著名的程序员，请根据问题描述、有效性评估及用户提供的解决方案，各自提供一个初步的且具体创新性的解决方案。
解决方案要包含基本的解决思路，使用的算法和数据描述，以及具体的实现步骤。

待解决的问题: {question}
对该问题描述的有效性评估：{commentq}
用户提供的解决方案：{raw_solution}

请一步步思考，确保解决方案能够解决上述问题，并转换为具体的代码实现。
你们三个的解决方案分别是（总长度不超过3000字）：'''

prompt_candidates = PromptTemplate(input_variables=['question', 'commentq', 'raw_solution'], template=template_candidates)
candidates_chain = LLMChain(llm=llm, prompt=prompt_candidates, output_key='candidates')

In [5]:
template_rewritea = '''你是一个专业的流程挖掘领域的专利代理，你的任务是帮助专利发明人完善发明点设计。
请根据待解决问题的描述，自动生成的解决方案和用户提供的解决方案，重新组合生成一个可以作为专利发明点的解决方案。
这个解决方案需要具体、详细，能够转换为代码实现，还要包含以下内容：
1，它的基本思路是什么？用到了哪些成熟算法和数据？请具体分析这个方法如何解决问题的。
2，它与一般的解决方法有什么不同？不管是算法上的不同，应用场景的不同，还是使用的数据不同都可以。
3，这些不同点对应的优势和价值是什么？如果创新点的价值不高，可以通过什么样的方法来提高？
4，它的具体解决步骤是什么？可以综合多个解决方案中的实需现，但要保证每个步骤都是必须的，而且能够解决全部问题。

待解决问题的描述: {question}
自动生成的解决方案: {candidates}
发明人提供的解决方案：{raw_solution}

请根据上述方案，在发明人提供的解决方案基础上，生成一个具备创新性的解决方案（总长度不超过2000字）：'''

prompt_rewritea = PromptTemplate(input_variables=['question', 'candidates', 'raw_solution'], template=template_rewritea)
rewritea_chain = LLMChain(llm=llm, prompt=prompt_rewritea, output_key='solution')

In [6]:
template_background = '''你是一个专业的流程挖掘领域的专利代理，帮助专利发明人设计发明点设计，并起草专利草稿。请根据以下待解决问题及解决方案的描述，起草专利的背景介绍部分。
专利的背景介绍要包含所解决问题所在的技术领域介绍，问题的简单介绍，以及这个问题所产生的不良影响。
待解决问题的描述: {question}
解决方案的描述: {solution}
请生成这个专利发明点的背景介绍（总长度不超过2000字）：'''

prompt_background = PromptTemplate(input_variables=['question', 'solution'], template=template_background)
background_chain = LLMChain(llm=llm, prompt=prompt_background, output_key='background')

In [7]:
template_claims = '''你是一个专业的流程挖掘和任务挖掘方面的专利代理，负责专利发明点设计和专利文档起草。请根据以下要解决的问题及解决方案，起草专利的权利要求部分。
在设计专利的权利要求时要一步步来，先判断解决方案的创新性和价值所在，然后再根据创新点来设计对应的权利要求。针对要解决的问题，先将解决方案重新描述为多个步骤，再判断每一个步骤
是否有必要存在，若是没有必要，则可以删除；对于必要存在的步骤，再考虑它是否具备创新性。若是不具备，则更新对应的步骤。只要其中一个步骤具备创新性即可。
可使用以下问题来引导生成过程：

要解决的问题: {question}
提供的解决方案: {solution}
请生成这个专利发明点的权利要求（总长度不超过4000字）：'''

prompt_claims = PromptTemplate(input_variables=['question', 'solution'], template=template_claims)
claims_chain = LLMChain(llm=llm, prompt=prompt_claims, output_key='claims')

In [8]:
template_explanation = '''你是一个专业的流程挖掘和任务挖掘方面的专利代理，负责专利发明点设计和专利文档起草。请根据解决的问题、解决方案及权利要求，
扩充权利要求部分的内容。扩充的方法是，先设计一个具体的应用场景，将权利要求转换为具体的实现步骤，并结合应用场景进行描述每一步的输入输出，以及中间的处理过程。
若是不同权利要求对应的操作步骤是相同的，则只需要在第一次给出描述，后面的则省略。
可使用这些问题来引导生成过程：它的主要算法或者技术是什么？需要什么样的软硬件运行环境？是否有多种实现方式？每种实现方式最简单的形式是什么样的？

要解决的问题: {question}
提供的解决方案: {solution}
独立权利要求: {claims}

请将权利要求扩展为具体的实现步骤，并给出相应的示例说明（总长度不超过2000字）：'''

prompt_explanation = PromptTemplate(input_variables=['question', 'solution', 'claims'], template=template_explanation)
explanation_chain = LLMChain(llm=llm, prompt=prompt_explanation, output_key='explanation')

In [9]:
from langchain.chains import SequentialChain

def generate(pno, question, solution, version=1):
    result = overall_chain({'raw_question': question, 'raw_solution': solution})

    with open(f'patents/patent-rpm-{pno}v{version}.txt', 'w') as f:
        f.write('>>> original question: \n' + result['raw_question'])  
        f.write('\n\n>>> original solution: \n' + result['raw_solution'])
        f.write('\n\n>>> revised question: \n' + result['question'])
        f.write('\n\n>>> commented question: \n' + result['commentq'])
        f.write('\n\n>>> solution candidates: \n' + result['candidates'])
        f.write('\n\n>>> final solution: \n' + result['solution'])
        f.write('\n\n>>> background: \n' + result['background'])
        f.write('\n\n>>> claims: \n' + result['claims'])
        f.write('\n\n>>> explanation: \n' + result['explanation'])

overall_chain = SequentialChain(
    chains=[rewriteq_chain, commentq_chain, candidates_chain, rewritea_chain, background_chain, claims_chain, explanation_chain],
    input_variables=["raw_question", "raw_solution"],
    # Here we return multiple variables
    output_variables=["question", "commentq", "candidates", "solution", "background", "claims", "explanation"],
    verbose=True)

In [14]:
import pandas as pd


version = 2
df = pd.read_excel('patents/rpm-patents-0628d.xlsx')
for i, row in df.iterrows():
    pno, question, solution = row['no.'], row['question'], row['solution']
    if pd.isna(question) or pd.isna(solution): continue
    print(pno, question)
    generate(pno, question, solution, version)

P73 使用GPT可以生成规模并不复杂的代码，包括RPA代码，一般的方法是设置一个比较详细的生成提示词，但是这样的生成过程难以控制，比如使用了不存在的库函数，或者缺少必要的数据等等，也难以保证生成的流程可以正常执行。


> Entering new SequentialChain chain...

> Finished chain.
P74 使用GPT可以生成能够正常运行的不太复杂的代码，但是没有方法可以控制生成的内容。在实际业务场景中，一段代码所能够调用的函数是范围限制的。任意生成的代码无法考虑这种限制，所以生成的代码即使可用，也无法直接使用，还需要人工修改，达不到节省人力的目的。


> Entering new SequentialChain chain...

> Finished chain.
P75 使用GPT来生成代码时，生成所需要的上下文有可能非常多，无法容纳在模型的窗口大小范围之内。常用的方法是先对上下文做摘要，然后使用摘要作为上下文来指导代码的生成。由于代码包含的细节非常多，这种摘要方式很容易丢失大量的细节。


> Entering new SequentialChain chain...

> Finished chain.
